In [1]:
import requests
import pandas as pd
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
import lxml
import cchardet
import os.path

In [2]:
cd ..

/Users/andyczeropski/Desktop/BAT_Analytics/v2


In [3]:
from scraper_tools import main as scrape

## Get List of Vehicle URL's from CSV

In [4]:
vehicle_urls_xls_fp = r'/Users/andyczeropski/Desktop/BAT_Analytics/v2/urls/makes_urls/vehicle_urls_final.csv'
vehicle_urls_lst = pd.read_csv(vehicle_urls_xls_fp)['url'].to_list()
print(f'Found {len(vehicle_urls_lst)} vehicle urls to scrape.')

Found 90753 vehicle urls to scrape.


## Scrape Data From URLs into CSV

In [5]:
# Read/create current vehicle data csv file
vehicle_data_fp = r'/Users/andyczeropski/Desktop/BAT_Analytics/v2/data/vehicle_data_v2.csv'
vehicle_data_error_fp = r'/Users/andyczeropski/Desktop/BAT_Analytics/v2/data/vehicle_data_errors_v2.csv'

# Check if vehicle data csv exists
if os.path.isfile(vehicle_data_fp):
    vehicle_data_df = pd.read_csv(vehicle_data_fp)
    vehicle_data_scraped_urls = vehicle_data_df['url'].to_list()
else:
    vehicle_data_scraped_urls = []

# Loop through all vehicle urls to scrape data, if url is already collected, we skip
urls = vehicle_urls_lst
for idx, url in enumerate(urls):
    try:
        if url not in vehicle_data_scraped_urls:
            # get vehicle html
            try:
                html = requests.get(url)
            except requests.exceptions.RequestException as e:  # This is the correct syntax
                print("URL is not correct")

            # Pull html data
            vehicle_data_soup = BeautifulSoup(html.content, 'lxml')

            vehicle_dict = scrape(vehicle_data_soup)

            vehicle_dict.pop('image') # Used for front end but not needed here
            vehicle_dict['url'] = url # add url for url checking in case program crashes mid scrape

            # Get columns from dict
            COLUMNS = [key for key, value in vehicle_dict.items()]

            # Make sure to wrap dict in list -> [dict]
            pd_db = pd.DataFrame.from_dict([vehicle_dict], orient='columns')

            pd_db.set_index('auction_year', inplace=True)
            if os.path.exists(vehicle_data_fp):
                pd_db.to_csv(vehicle_data_fp, mode='a', header=False)
            else:
                pd_db.to_csv(vehicle_data_fp, mode='a', header=True)
            
    except Exception as e:
        print('ERROR: data not collected.')
        print(e)
        # Collect error url data into .csv
        if os.path.exists(vehicle_data_error_fp):
            pd_db = pd.DataFrame([{'url' : url}])
            pd_db.to_csv(vehicle_data_error_fp, mode='a', header=False)
        else:
            pd_db = pd.DataFrame([{'url' : url}])
            pd_db.to_csv(vehicle_data_error_fp, mode='a', header=True)

    # Print out every 100 completed vehicles
    if idx % 100 == 0:
        print(f'---Completed {idx} out of {len(vehicle_urls_lst)} vehicles---')


---Completed 0 out of 90753 vehicles---
---Completed 100 out of 90753 vehicles---
---Completed 200 out of 90753 vehicles---
---Completed 300 out of 90753 vehicles---
---Completed 400 out of 90753 vehicles---
---Completed 500 out of 90753 vehicles---
---Completed 600 out of 90753 vehicles---
---Completed 700 out of 90753 vehicles---
---Completed 800 out of 90753 vehicles---
---Completed 900 out of 90753 vehicles---
---Completed 1000 out of 90753 vehicles---
---Completed 1100 out of 90753 vehicles---
---Completed 1200 out of 90753 vehicles---
---Completed 1300 out of 90753 vehicles---
---Completed 1400 out of 90753 vehicles---
---Completed 1500 out of 90753 vehicles---
---Completed 1600 out of 90753 vehicles---
---Completed 1700 out of 90753 vehicles---
---Completed 1800 out of 90753 vehicles---
---Completed 1900 out of 90753 vehicles---
---Completed 2000 out of 90753 vehicles---
---Completed 2100 out of 90753 vehicles---
---Completed 2200 out of 90753 vehicles---
---Completed 2300 out o